Time series.Sequence of observations collected over time - usually at regular intervals
- cannot be randomly shaffled because its ordinal data(categorical data)
#### componets
1. Trend
2. Seasonality repeating patterns
3. Cyclic patterns

Time series = Trend + seasonality + noise
- ML does not understand time convert to features
- lag features
#### Rolling statistics
- capture local trends
- rolling mean (7days)
- rolling standard deviation

In [4]:
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error # get prediction error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor # This is due to non linear patterns
from sklearn.svm import SVR # help to capture non linear patterns avoid over fitting
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('callls.csv')
df.head()

,StartDateTime,UserId,CallType,disconnectType,Total_Handle_Time,Total_Hold_Time,Total_Talk_Time,Unnamed: 7,Unnamed: 8
0,1/1/2023 5:01:42 AM,da7fadb4-cad9-4062-8271-bbc999ac1275,Inbound_Queue,endpoint,1188.162,0.000,1075.162,NaN,NaN
1,1/1/2023 5:11:53 AM,af512203-3ea2-48d5-bbf5-e46ae915d4d8,Inbound_Queue,peer,396.923,0.000,216.923,NaN,NaN
2,1/1/2023 5:24:17 AM,6b633a32-ec20-48f4-91dd-503d459eda1f,Inbound_Queue,peer,537.893,173.363,361.544,NaN,NaN
3,1/1/2023 6:01:03 AM,63bb6f93-004c-4a79-8d01-7de24f75241b,Inbound_Queue,peer,266.625,0.000,263.625,NaN,NaN
4,1/1/2023 6:04:02 AM,e47ca893-1edc-49e9-b0bd-58cde8182443,Inbound_Queue,peer,2545.198,856.591,1674.901,NaN,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135859 entries, 0 to 135858
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   StartDateTime      135859 non-null  object 
 1   UserId             135859 non-null  object 
 2   CallType           135859 non-null  object 
 3   disconnectType     135830 non-null  object 
 4   Total_Handle_Time  135859 non-null  float64
 5   Total_Hold_Time    135859 non-null  float64
 6   Total_Talk_Time    135859 non-null  float64
 7   Unnamed: 7         0 non-null       float64
 8   Unnamed: 8         0 non-null       float64
dtypes: float64(5), object(4)
memory usage: 9.3+ MB


In [ ]:
df= df.drop(columns=["Unnamed: 7","Unnamed: 8"])# remove columns
df.head()

,StartDateTime,UserId,CallType,disconnectType,Total_Handle_Time,Total_Hold_Time,Total_Talk_Time
0,1/1/2023 5:01:42 AM,da7fadb4-cad9-4062-8271-bbc999ac1275,Inbound_Queue,endpoint,1188.162,0.000,1075.162
1,1/1/2023 5:11:53 AM,af512203-3ea2-48d5-bbf5-e46ae915d4d8,Inbound_Queue,peer,396.923,0.000,216.923
2,1/1/2023 5:24:17 AM,6b633a32-ec20-48f4-91dd-503d459eda1f,Inbound_Queue,peer,537.893,173.363,361.544
3,1/1/2023 6:01:03 AM,63bb6f93-004c-4a79-8d01-7de24f75241b,Inbound_Queue,peer,266.625,0.000,263.625
4,1/1/2023 6:04:02 AM,e47ca893-1edc-49e9-b0bd-58cde8182443,Inbound_Queue,peer,2545.198,856.591,1674.901


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135859 entries, 0 to 135858
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   StartDateTime      135859 non-null  object 
 1   UserId             135859 non-null  object 
 2   CallType           135859 non-null  object 
 3   disconnectType     135830 non-null  object 
 4   Total_Handle_Time  135859 non-null  float64
 5   Total_Hold_Time    135859 non-null  float64
 6   Total_Talk_Time    135859 non-null  float64
dtypes: float64(3), object(4)
memory usage: 7.3+ MB


In [11]:
df['StartDateTime']=pd.to_datetime(df['StartDateTime'],errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135859 entries, 0 to 135858
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   StartDateTime      79917 non-null   datetime64[ns]
 1   UserId             135859 non-null  object        
 2   CallType           135859 non-null  object        
 3   disconnectType     135830 non-null  object        
 4   Total_Handle_Time  135859 non-null  float64       
 5   Total_Hold_Time    135859 non-null  float64       
 6   Total_Talk_Time    135859 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 7.3+ MB


In [12]:
df = df.sort_values('StartDateTime').reset_index(drop=True)

In [14]:
target = 'Total_Handle_Time'

In [15]:
df= df.set_index('StartDateTime')
ts = df.resample("10T").agg({
    'Total_Handle_Time':'mean',
    'Total_Hold_Time':'mean',
    'Total_Talk_Time':'mean',

})

C:\Users\Admin\AppData\Local\Temp\ipykernel_45140\2799041747.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ts = df.resample("10T").agg({


In [16]:
ts.head()

,Total_Handle_Time,Total_Hold_Time,Total_Talk_Time
StartDateTime,,,
2023-01-01 05:00:00,1188.162,0.000,1075.162
2023-01-01 05:10:00,396.923,0.000,216.923
2023-01-01 05:20:00,537.893,173.363,361.544
2023-01-01 05:30:00,NaN,NaN,NaN
2023-01-01 05:40:00,NaN,NaN,NaN


In [17]:
#Model or Ml does not know the past , The lag give the ML memory 
#important to remove null values
ts = ts.dropna()

In [18]:
ts.head()

,Total_Handle_Time,Total_Hold_Time,Total_Talk_Time
StartDateTime,,,
2023-01-01 05:00:00,1188.162000,0.000000,1075.162000
2023-01-01 05:10:00,396.923000,0.000000,216.923000
2023-01-01 05:20:00,537.893000,173.363000,361.544000
2023-01-01 06:00:00,1279.849333,292.399333,981.214667
2023-01-01 06:10:00,1486.984000,291.098500,1060.135500


In [20]:
ts['lag_1']= ts[target].shift(1)
ts['lag_2']= ts[target].shift(2)
ts['lag_3']= ts[target].shift(3)
ts


,Total_Handle_Time,Total_Hold_Time,Total_Talk_Time,lag_1,lag_2,lag_3
StartDateTime,,,,,,
2023-01-01 05:00:00,1188.162000,0.000000,1075.162000,NaN,NaN,NaN
2023-01-01 05:10:00,396.923000,0.000000,216.923000,1188.162000,NaN,NaN
2023-01-01 05:20:00,537.893000,173.363000,361.544000,396.923000,1188.162000,NaN
2023-01-01 06:00:00,1279.849333,292.399333,981.214667,537.893000,396.923000,1188.162000
2023-01-01 06:10:00,1486.984000,291.098500,1060.135500,1279.849333,537.893000,396.923000
...,...,...,...,...,...,...
2023-03-15 11:10:00,1239.948590,204.841282,899.815282,1341.959943,1239.101512,1676.881437
2023-03-15 11:20:00,1292.257313,258.367896,910.082083,1239.948590,1341.959943,1239.101512
2023-03-15 11:30:00,1558.809091,273.606000,1133.476545,1292.257313,1239.948590,1341.959943


In [22]:
#Rolling statistics
ts['rolling_mean_3'] = ts[target].rolling(3).mean()
ts['rolling_std_3']=ts[target].rolling(3).std()
ts


,Total_Handle_Time,Total_Hold_Time,Total_Talk_Time,lag_1,lag_2,lag_3,rolling_mean_3,rolling_std_3
StartDateTime,,,,,,,,
2023-01-01 05:00:00,1188.162000,0.000000,1075.162000,NaN,NaN,NaN,NaN,NaN
2023-01-01 05:10:00,396.923000,0.000000,216.923000,1188.162000,NaN,NaN,NaN,NaN
2023-01-01 05:20:00,537.893000,173.363000,361.544000,396.923000,1188.162000,NaN,707.659333,422.054789
2023-01-01 06:00:00,1279.849333,292.399333,981.214667,537.893000,396.923000,1188.162000,738.221778,474.329466
2023-01-01 06:10:00,1486.984000,291.098500,1060.135500,1279.849333,537.893000,396.923000,1101.575444,499.028673
...,...,...,...,...,...,...,...,...
2023-03-15 11:10:00,1239.948590,204.841282,899.815282,1341.959943,1239.101512,1676.881437,1273.670015,59.142329
2023-03-15 11:20:00,1292.257313,258.367896,910.082083,1239.948590,1341.959943,1239.101512,1291.388615,51.011224
2023-03-15 11:30:00,1558.809091,273.606000,1133.476545,1292.257313,1239.948590,1341.959943,1363.671664,171.005883


In [23]:
ts.dropna()

,Total_Handle_Time,Total_Hold_Time,Total_Talk_Time,lag_1,lag_2,lag_3,rolling_mean_3,rolling_std_3
StartDateTime,,,,,,,,
2023-01-01 06:00:00,1279.849333,292.399333,981.214667,537.893000,396.923000,1188.162000,738.221778,474.329466
2023-01-01 06:10:00,1486.984000,291.098500,1060.135500,1279.849333,537.893000,396.923000,1101.575444,499.028673
2023-01-01 06:20:00,656.413000,0.000000,476.413000,1486.984000,1279.849333,537.893000,1141.082111,432.324303
2023-01-01 06:30:00,1156.589000,388.979000,587.610000,656.413000,1486.984000,1279.849333,1099.995333,418.167644
2023-01-01 06:40:00,806.433000,133.788400,509.644600,1156.589000,656.413000,1486.984000,873.145000,256.674650
...,...,...,...,...,...,...,...,...
2023-03-15 11:10:00,1239.948590,204.841282,899.815282,1341.959943,1239.101512,1676.881437,1273.670015,59.142329
2023-03-15 11:20:00,1292.257313,258.367896,910.082083,1239.948590,1341.959943,1239.101512,1291.388615,51.011224
2023-03-15 11:30:00,1558.809091,273.606000,1133.476545,1292.257313,1239.948590,1341.959943,1363.671664,171.005883
